In [1]:
import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import cm
# from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
# from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [2]:
class Individual(object):
    
    def __init__(self, m_rate, pack_size, genotype=[]):
        
        if not genotype:
            self.genotype = np.random.randint(2,size=pack_size)
        else:
            self.genotype = np.array(genotype)
            
        self.m_rate = m_rate
        
#         self.eval_func = evaluation_function
#         self.fitness = self.evaluate()
        
    def evaluate(self):
        self.fitness = self.eval_func(self.phenotype())

    def mutate(self):
        r = np.random.random()
        for i in range(len(self.genotype)):
            if r < self.m_rate:
                self.genotype[i] = 0 if self.genotype[i] == 1 else 1
    
    def new_ind(self, genotype):
        return Individual(self.m_rate, len(genotype), genotype)
    
    def recombine(self,mate):
        ind1 = list(self.genotype)
        ind2 = list(mate.genotype)
        
        r = np.random.randint(1,len(ind1)-1)
        
        new_ind1 = ind1[:r] + ind2[r:]
        new_ind2 = ind2[:r] + ind1[r:]
        
        new_ind1 = self.new_ind(new_ind1)
        new_ind2 = self.new_ind(new_ind2)
        
        return new_ind1,new_ind2
        
#     def phenotype(self):
#         x1 = -5 + decimal(self.genotype[:17])*10/np.float(2**17 - 1 )
#         x2 = -5 + decimal(self.genotype[17:34])*10/np.float(2**17 - 1 )

#         return (x1,x2)

# def decimal(bitstring):
#     init = 0
#     for bit in bitstring:
#         init = (init << 1) | bit
#     return init 


Classe que gerencia a populacao

In [3]:
class Population(object):
    ' Manages GA population '''

    def __init__(self, popsize, individual_class, mutation_rate, recombination_rate, t_max, V, W, MAX, roulette):
        self.popsize = popsize
        self.population = [individual_class(mutation_rate, len(V)) for _ in range(popsize)]
        self.mutation_rate = mutation_rate
        self.recombination_rate = recombination_rate
        self.selected = []
        self.t = 0
        self.t_max = t_max
        self.V = V
        self.W = W
        self.MAX = MAX
        self.roulette = roulette
        
    def selection(self):
        
        pop_fitness = np.array([self.getFitness(ind) for ind in self.population])
        
        self.selectRoulette(pop_fitness) if self.roulette else self.selectTournament(pop_fitness)
        
        
    def selectRoulette(self, pop_fitness):
        
        pop_fitness = (pop_fitness - np.min(pop_fitness))/(np.max(pop_fitness) - np.min(pop_fitness))
        pop_fitness = pop_fitness / np.sum(pop_fitness)
        
        comu_fitness = np.cumsum(pop_fitness)
        ## alternative sampling
        ## x = np.random.choice(self.population,p=pop_fitness,size=self.popsize)
        
        new_pop = []
        while len(new_pop) < self.popsize:
            r = np.random.uniform()
            for ind in range(self.popsize):
                if comu_fitness[ind] >= r:
                    new_pop.append(ind)
        
        self.selected = [self.population[i] for i in new_pop]
        
    def selectTournament(self, pop_fitness):
        
        k = 0.75
        new_pop = []
        while len(new_pop) < self.popsize:
            x = list(range(self.popsize))
            ind1 = np.random.choice(x)
            x.remove(ind1)
            ind2 = np.random.choice(x)
                
            fit1 = pop_fitness[ind1]
            fit2 = pop_fitness[ind2]
            best_worst = (ind1, ind2) if fit1 > fit2 else (ind2, ind1)
            
            r = np.random.uniform()
            
            if r < k:
                new_pop.append(best_worst[0])
            else:
                new_pop.append(best_worst[1])
        
        self.selected = [self.population[i] for i in new_pop]
        

    def mutation(self):
        for ind in self.selected:
            ind.mutate()
            
        
    def termination(self):
        
        self.t += 1
        return self.t < self.t_max
        
    def recombination(self):
        
        for i in range(self.popsize):
            r = np.random.uniform()
            if r < self.recombination_rate:
                x = list(range(self.popsize))
                x.remove(i)
                k = np.random.choice(x)
                
                ind1 = self.selected[i]
                ind2 = self.selected[k]
                
                ind3, ind4 = ind1.recombine(ind2)
                
                self.selected[i] = ind3
                self.selected[k] = ind4
            
    def getFitness(self, ind):
        weight = 0
        volume = 0
        ones = []
        
        for i in range(len(ind.genotype)):
            if ind.genotype[i] == 1:
                volume += self.V[i]
                weight += self.W[i]
                ones += [i]

        while (volume > self.MAX):
            randIndex = np.random.randint(0,len(ones)-1)
#             randIndex = random.randint(0, len(ones)-1)
            
            ind.genotype[ones[randIndex]] = 0
            volume -= self.V[ones[randIndex]]
            weight -= self.W[ones[randIndex]]
            ones.remove(ones[randIndex])

        return weight
    
    def next_generation(self):
        self.population = self.selected
        
    def getElite(self):
        pop_fitness = np.array([self.getFitness(ind) for ind in self.population])
        return pop_fitness

    def bookkeeping(self):
        pass    


In [36]:
from time import time

values = [84,83,43,4,44,6,82,92,25,83,56,18,58,14,48,70,96,32,68,92]
weights = [91,72,90,46,55,8,35,75,61,15,77,40,63,75,29,75,17,78,40,44]
maxVolume = 879

def run_ga():
    
    pop = Population(popsize = 10*len(values),
                     individual_class = Individual,
                     mutation_rate = 0.2,
                     recombination_rate = 0.8,
                     t_max = 100*len(values),
                     V = values,
                     W = weights,
                     MAX = maxVolume,
                     roulette = True
                     )
    
    setTime = 0
    while pop.termination():
        t1 = time()
        pop.selection()
        pop.recombination()
        pop.mutation()
        pop.next_generation()
        t2 = time()    
        setTime += t2-t1
        
    print(str(setTime/pop.t_max))
    
    return pop.getElite()
    

for x in range(0, 4):    
    ans = run_ga()
    print(str(max(ans)))

0.009277240991592408
833
0.009342154622077941
879
0.009199665904045105
857
0.009212775111198425
915


In [5]:
# ans

In [6]:
# low 1 - 1024
# values = [92,4,43,83,84,68,92,82,6,44,32,18,56,83,25,96,70,48,14,58]
# weights = [44,46,90,72,91,40,75,35,8,54,78,40,77,15,61,17,75,29,75,63]
# maxVolume = 878

In [7]:
# low 2 - 35
# values = [6,5,9,7]
# weights = [9,11,13,15]
# maxVolume = 20

In [8]:
# low 3 - 107
# values = [31,10,20,19,4,3,6]
# weights = [70,20,39,37,7,5,10]
# maxVolume = 50

In [9]:
# # low 4 - 1025
# values = [84,83,43,4,44,6,82,92,25,83,56,18,58,14,48,70,96,32,68,92]
# weights = [91,72,90,46,55,8,35,75,61,15,77,40,63,75,29,75,17,78,40,44]
# maxVolume = 879

In [10]:
# large 1 - 9147
# values = [485,326,248,421,322,795,43,845,955,252,9,901,122,94,738,574,715,882,367,984,299,433,682,72,874,138,856,145,995,529,199,277,97,719,242,107,122,70,98,600,645,267,972,895,213,748,487,923,29,674,540,554,467,46,710,553,191,724,730,988,90,340,549,196,865,678,570,936,722,651,123,431,508,585,853,642,992,725,286,812,859,663,88,179,187,619,261,846,192,261,514,886,530,849,294,799,391,330,298,790]
# weights = [94,506,416,992,649,237,457,815,446,422,791,359,667,598,7,544,334,766,994,893,633,131,428,700,617,874,720,419,794,196,997,116,908,539,707,569,537,931,726,487,772,513,81,943,58,303,764,536,724,789,479,142,339,641,196,494,66,824,208,711,800,314,289,401,466,689,833,225,244,849,113,379,361,65,486,686,286,889,24,491,891,90,181,214,17,472,418,419,356,682,306,201,385,952,500,194,737,324,992,224]
# maxVolume = 995

In [11]:
# large 2 - 11238
# values = [485,326,248,421,322,795,43,845,955,252,9,901,122,94,738,574,715,882,367,984,299,433,682,72,874,138,856,145,995,529,199,277,97,719,242,107,122,70,98,600,645,267,972,895,213,748,487,923,29,674,540,554,467,46,710,553,191,724,730,988,90,340,549,196,865,678,570,936,722,651,123,431,508,585,853,642,992,725,286,812,859,663,88,179,187,619,261,846,192,261,514,886,530,849,294,799,391,330,298,790,275,826,72,866,951,748,685,956,564,183,400,721,207,323,611,116,109,795,343,862,685,10,881,984,403,360,449,541,272,877,359,707,308,770,30,208,311,100,939,422,785,370,989,969,143,972,28,61,638,348,347,66,391,638,295,826,196,449,855,143,487,140,564,615,135,564,360,793,163,859,760,711,662,159,660,268,948,315,676,341,689,894,706,490,478,671,932,899,237,187,472,772,98,906,911,635,225,823,164,343]
# weights = [94,506,416,992,649,237,457,815,446,422,791,359,667,598,7,544,334,766,994,893,633,131,428,700,617,874,720,419,794,196,997,116,908,539,707,569,537,931,726,487,772,513,81,943,58,303,764,536,724,789,479,142,339,641,196,494,66,824,208,711,800,314,289,401,466,689,833,225,244,849,113,379,361,65,486,686,286,889,24,491,891,90,181,214,17,472,418,419,356,682,306,201,385,952,500,194,737,324,992,224,260,97,210,649,919,63,958,804,518,428,537,346,153,971,662,197,91,529,126,747,469,770,934,723,895,568,172,958,383,308,970,583,48,930,569,3,20,609,887,825,930,904,241,379,376,962,889,443,216,338,160,406,159,31,204,420,153,425,331,565,838,9,918,533,232,957,591,576,746,377,858,86,434,558,279,840,735,574,126,912,739,821,625,917,201,993,149,52,759,267,256,783,117,516,180,25,380,712,266,216]
# maxVolume = 1008

In [12]:
# large 3 - 1514
# values = [485,326,248,421,322,795,43,845,955,252,9,901,122,94,738,574,715,882,367,984,299,433,682,72,874,138,856,145,995,529,199,277,97,719,242,107,122,70,98,600,645,267,972,895,213,748,487,923,29,674,540,554,467,46,710,553,191,724,730,988,90,340,549,196,865,678,570,936,722,651,123,431,508,585,853,642,992,725,286,812,859,663,88,179,187,619,261,846,192,261,514,886,530,849,294,799,391,330,298,790]
# weights = [482,257,286,517,404,713,45,924,873,160,1,838,40,58,676,627,766,862,405,923,379,461,612,133,813,97,908,165,996,623,220,298,157,723,144,48,129,148,35,644,632,272,1040,977,312,778,567,965,1,616,569,628,493,76,733,575,288,775,723,912,64,354,565,210,922,775,566,934,626,742,194,485,483,617,876,653,896,652,220,727,900,563,56,157,280,537,284,920,124,239,459,931,504,910,382,795,485,351,289,865]
# maxVolume = 995

In [13]:
# large 4 - 7117
# values = [485,94,326,506,248,416,421,992,322,649,795,237,43,457,845,815,955,446,252,422,9,791,901,359,122,667,94,598,738,7,574,544,715,334,882,766,367,994,984,893,299,633,433,131,682,428,72,700,874,617,138,874,856,720,145,419,995,794,529,196,199,997,277,116,97,908,719,539,242,707,107,569,122,537,70,931,98,726,600,487,645,772,267,513,972,81,895,943,213,58,748,303,487,764,923,536,29,724,674,789,540,479,554,142,467,339,46,641,710,196,553,494,191,66,724,824,730,208,988,711,90,800,340,314,549,289,196,401,865,466,678,689,570,833,936,225,722,244,651,849,123,113,431,379,508,361,585,65,853,486,642,686,992,286,725,889,286,24,812,491,859,891,663,90,88,181,179,214,187,17,619,472,261,418,846,419,192,356,261,682,514,306,886,201,530,385,849,952,294,500,799,194,391,737,330,324,298,992,790,224,275,260,826,97,72,210,866,649,951,919,748,63,685,958,956,804,564,518,183,428,400,537,721,346,207,153,323,971,611,662,116,197,109,91,795,529,343,126,862,747,685,469,10,770,881,934,984,723,403,895,360,568,449,172,541,958,272,383,877,308,359,970,707,583,308,48,770,930,30,569,208,3,311,20,100,609,939,887,422,825,785,930,370,904,989,241,969,379,143,376,972,962,28,889,61,443,638,216,348,338,347,160,66,406,391,159,638,31,295,204,826,420,196,153,449,425,855,331,143,565,487,838,140,9,564,918,615,533,135,232,564,957,360,591,793,576,163,746,859,377,760,858,711,86,662,434,159,558,660,279,268,840,948,735,315,574,676,126,341,912,689,739,894,821,706,625,490,917,478,201,671,993,932,149,899,52,237,759,187,267,472,256,772,783,98,117,906,516,911,180,635,25,225,380,823,712,164,266,343,216,732,448,502,541,740,664,576,954,612,726,902,772,454,531,411,943,973,750,703,327,850,917,77,5,220,113,802,913,403,791,181,998,10,859,525,345,919,431,668,675,527,833,462,438,291,523,605,916,457,420,405,115,417,660,279,261,685,772,596,388,307,764,224,843,322,206,840,407,975,639,401,852,91,542,327,60,330,757,182,82,603,637,793,93,615,614,733,136,864,187,16,102,863,972,987,163,306,501,34,477]
# weights = [585,194,426,606,348,516,521,1092,422,749,895,337,143,557,945,915,1055,546,352,522,109,891,1001,459,222,767,194,698,838,107,674,644,815,434,982,866,467,1094,1084,993,399,733,533,231,782,528,172,800,974,717,238,974,956,820,245,519,1095,894,629,296,299,1097,377,216,197,1008,819,639,342,807,207,669,222,637,170,1031,198,826,700,587,745,872,367,613,1072,181,995,1043,313,158,848,403,587,864,1023,636,129,824,774,889,640,579,654,242,567,439,146,741,810,296,653,594,291,166,824,924,830,308,1088,811,190,900,440,414,649,389,296,501,965,566,778,789,670,933,1036,325,822,344,751,949,223,213,531,479,608,461,685,165,953,586,742,786,1092,386,825,989,386,124,912,591,959,991,763,190,188,281,279,314,287,117,719,572,361,518,946,519,292,456,361,782,614,406,986,301,630,485,949,1052,394,600,899,294,491,837,430,424,398,1092,890,324,375,360,926,197,172,310,966,749,1051,1019,848,163,785,1058,1056,904,664,618,283,528,500,637,821,446,307,253,423,1071,711,762,216,297,209,191,895,629,443,226,962,847,785,569,110,870,981,1034,1084,823,503,995,460,668,549,272,641,1058,372,483,977,408,459,1070,807,683,408,148,870,1030,130,669,308,103,411,120,200,709,1039,987,522,925,885,1030,470,1004,1089,341,1069,479,243,476,1072,1062,128,989,161,543,738,316,448,438,447,260,166,506,491,259,738,131,395,304,926,520,296,253,549,525,955,431,243,665,587,938,240,109,664,1018,715,633,235,332,664,1057,460,691,893,676,263,846,959,477,860,958,811,186,762,534,259,658,760,379,368,940,1048,835,415,674,776,226,441,1012,789,839,994,921,806,725,590,1017,578,301,771,1093,1032,249,999,152,337,859,287,367,572,356,872,883,198,217,1006,616,1011,280,735,125,325,480,923,812,264,366,443,316,832,548,602,641,840,764,676,1054,712,826,1002,872,554,631,511,1043,1073,850,803,427,950,1017,177,105,320,213,902,1013,503,891,281,1098,110,959,625,445,1019,531,768,775,627,933,562,538,391,623,705,1016,557,520,505,215,517,760,379,361,785,872,696,488,407,864,324,943,422,306,940,507,1075,739,501,952,191,642,427,160,430,857,282,182,703,737,893,193,715,714,833,236,964,287,116,202,963,1072,1087,263,406,601,134,577]
# maxVolume = 2517